<a href="https://colab.research.google.com/github/Lomesh2000/Apple-and-Orange-detector-using-YOLOV5-/blob/Lomesh2000/apple_and_orange_detection_and_counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import cv2
import numpy as np
from tqdm import tqdm
import argparse
import fileinput

os.chdir('/content/drive/MyDrive/projects/Artenal/yolov5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/projects/Artenal')

## Cloning yolov5 github repository .

In [ ]:
! git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 12589, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12589 (delta 6), reused 8 (delta 3), pack-reused 12575
Receiving objects: 100% (12589/12589), 12.32 MiB | 6.31 MiB/s, done.
Resolving deltas: 100% (8662/8662), done.


In [ ]:
! dir

Artenal.ipynb  yolov5


In [ ]:
% cd yolov5/

/content/drive/MyDrive/projects/Artenal/yolov5


In [ ]:
! dir

CONTRIBUTING.md  export.py   models	       setup.cfg       utils
data		 hubconf.py  README.md	       train.py        val.py
detect.py	 LICENSE     requirements.txt  tutorial.ipynb


### installing required modules for yolo to work

In [ ]:
% pip install -r requirements.txt

## Getting data from kaggle using kaggle API

In [ ]:
!mkdir ~/.kaggle

In [ ]:
! cp /content/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d balraj98/apple2orange-dataset

 99% 74.0M/74.8M [00:00<00:00, 148MB/s]
100% 74.8M/74.8M [00:00<00:00, 128MB/s]


In [ ]:
! mkdir data

In [ ]:
! unzip '/content/drive/MyDrive/projects/Artenal/apple2orange-dataset.zip' -d '/content/drive/MyDrive/projects/Artenal/data'

In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-f option requires 1 argument


In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
! pip install utils
import torch

from IPython.display import Image  # for displaying images
#from utils.downloads import gdrive_download  # for downloading models/datasets
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Using torch 1.12.0+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


/bin/bash: ./: Is a directory


In [ ]:
label_path='/content/drive/MyDrive/projects/Artenal/data/OID/csv_folder/Dataset/train/Apple_Orange/Label'

os.listdir(label_path)[:5]

['5f19841b492291a4.txt',
 '99df5d6584825c98.txt',
 '89670599b2f5f294.txt',
 'b22ef4542a98b712.txt',
 '07fb5124cb5d0296.txt']

In [ ]:
def annotation_acc_to_yolo(file):
  file_path=os.path.join(label_path,file)

  with open(file_path,'r+') as f:
    para = f.read()

    before=para.split()[0]

    para=para.replace(before,['0' if before=='This' else '1'][0])



  with open(file_path,'w') as f:
    f.write(para)    

In [ ]:
for file in os.listdir(label_path):

  annotation_acc_to_yolo(file)

In [ ]:
len(os.listdir(label_path))

1936

## function that turns XMin, YMin, XMax, YMax coordinates to normalized yolo format

In [ ]:


def convert(filename_str, coords):
    os.chdir("..")
    image = cv2.imread(filename_str + ".jpg")
    coords[2] -= coords[0]
    coords[3] -= coords[1]
    x_diff = int(coords[2]/2)
    y_diff = int(coords[3]/2)
    coords[0] = coords[0]+x_diff
    coords[1] = coords[1]+y_diff
    coords[0] /= int(image.shape[1])
    coords[1] /= int(image.shape[0])
    coords[2] /= int(image.shape[1])
    coords[3] /= int(image.shape[0])
    os.chdir("Label")
    return coords

ROOT_DIR = os.getcwd()

# create dict to map class names to numbers for yolo
classes = {}
with open("classes.txt", "r") as myFile:
    for num, line in enumerate(myFile, 0):
        line = line.rstrip("\n")
        classes[line] = num
    myFile.close()
# step into dataset directory
os.chdir(os.path.join("OID", "Dataset"))
DIRS = os.listdir(os.getcwd())

# for all train, validation and test folders
for DIR in DIRS:
    if os.path.isdir(DIR):
        os.chdir(DIR)
        print("Currently in subdirectory:", DIR)
        
        CLASS_DIRS = os.listdir(os.getcwd())
        # for all class folders step into directory to change annotations
        for CLASS_DIR in CLASS_DIRS:
            if os.path.isdir(CLASS_DIR):
                os.chdir(CLASS_DIR)
                print("Converting annotations for class: ", CLASS_DIR)
                
                # Step into Label folder where annotations are generated
                os.chdir("Label")

                for filename in tqdm(os.listdir(os.getcwd())):
                    filename_str = str.split(filename, ".")[0]
                    if filename.endswith(".txt"):
                        annotations = []
                        with open(filename) as f:
                            for line in f:
                                for class_type in classes:
                                    line = line.replace(class_type, str(classes.get(class_type)))
                                labels = line.split()
                                print(labels)
                                coords = np.asarray([float(labels[1]), float(labels[2]), float(labels[3]), float(labels[4])])
                                coords = convert(filename_str, coords)
                                labels[1], labels[2], labels[3], labels[4] = coords[0], coords[1], coords[2], coords[3]
                                newline = str(labels[0]) + " " + str(labels[1]) + " " + str(labels[2]) + " " + str(labels[3]) + " " + str(labels[4])
                                line = line.replace(line, newline)
                                annotations.append(line)
                            f.close()
                        os.chdir("..")
                        with open(filename, "w") as outfile:
                            for line in annotations:
                                outfile.write(line)
                                outfile.write("\n")
                            outfile.close()
                        os.chdir("Label")
                os.chdir("..")
                os.chdir("..")
        os.chdir("..")

In [ ]:
! mv '/content/drive/MyDrive/projects/Artenal/data/OID' '/content/drive/MyDrive/projects/Artenal/yolov5'

In [ ]:
%cd '/content/drive/MyDrive/projects/Artenal/yolov5'

/content/drive/MyDrive/projects/Artenal/yolov5


##  Training yolo model on our dataset

In [ ]:
!python train.py --img 416 --batch 16 --epochs 80 --data my_classes.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_results --cache --device 0

train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=my_classes.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 7 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-297-g72a81e7 Python-3.7.13 torch-1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1

## getting required results from the model using detect.py file

In [ ]:
!python detect.py --weights yolov5s.pt --img 416 --conf 0.4 --source '/content/o a.jpg' --save-conf

## Using pytorch for loading model and getting required results i.e. no of each fruit and thier respective cordinates 

In [ ]:
import cv2
import torch
from PIL import Image

# Model
model = torch.hub.load('/content/drive/MyDrive/projects/Artenal/yolov5', 'custom', source='local', path = '/content/drive/MyDrive/projects/Artenal/yolov5/yolov5s.pt', force_reload = True)

# Images
# for f in ['zidane.jpg', 'bus.jpg']:  # download 2 images
#     print(f'Downloading {f}...')
#     torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/' + f, f)
img1 = Image.open('/content/apples-and-oranges.jpg')  # PIL image
#img2 = cv2.imread('bus.jpg')[:, :, ::-1]  # OpenCV image (BGR to RGB)
#imgs = [img1, img2]  # batched list of images

# Inference
results = model(img1, size=640)  # includes NMS

# Results
results.print()  # print results to screen
results.show()  # display results
results.save()  # save as results1.jpg, results2.jpg... etc.

# Data
print('\n', results.xyxy[0])  # print img1 predictions in image coordinates
#          x1 (pixels)  y1 (pixels)  x2 (pixels)  y2 (pixels)   confidence        class
# tensor([[7.47613e+02, 4.01168e+01, 1.14978e+03, 7.12016e+02, 8.71210e-01, 0.00000e+00],
#         [1.17464e+02, 1.96875e+02, 1.00145e+03, 7.11802e+02, 8.08795e-01, 0.00000e+00],
#         [4.23969e+02, 4.30401e+02, 5.16833e+02, 7.20000e+02, 7.77376e-01, 2.70000e+01],
#         [9.81310e+02, 3.10712e+02, 1.03111e+03, 4.19273e+02, 2.86850e-01, 2.70000e+01]])

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: torch!=1.12.0,>=1.7.0 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113

requirements: torchvision!=0.13.0,>=0.8.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+

image 1/1: 282x425 6 apples, 6 oranges
Speed: 67.6ms pre-process, 7629.5ms inference, 57.5ms NMS per image at shape (1, 3, 448, 640)

 tensor([[2.06056e+02, 1.21301e+02, 3.36540e+02, 2.53358e+02, 7.17606e-01, 4.90000e+01],
        [3.37305e+02, 1.12750e+02, 4.24972e+02, 2.48825e+02, 6.85552e-01, 4.90000e+01],
        [4.59522e-02, 3.77236e+01, 9.07212e+01, 1.07953e+02, 6.72831e-01, 4.70000e+01],
        [3.20356e+02, 3.50102e+01, 4.25000e+02, 1.51670e+02, 6.72331e-01, 4.90000e+01],
        [2.07354e+02, 6.86056e+01, 3.33044e+02, 1.54325e+02, 6.14948e-01, 4.90000e+01],
        [0.00000e+00, 1.17340e+02, 7.55246e+01, 2.55065e+02, 5.95355e-01, 4.70000e+01],
        [9.07156e+01, 3.80929e+01, 2.05972e+02, 9.97866e+01, 5.84609e-01, 4.70000e+01],
        [9.19976e+01, 7.06963e+01, 2.13553e+02, 1.50970e+02, 5.44398e-01, 4.70000e+01],
        [7.06515e+01, 1.05944e+02, 2.03213e+02, 2.57297e+02, 5.07843e-01, 4.70000e+01],
        [2.07835e+02, 4.74537e+01, 3.21394e+02, 1.18247e+02, 4.51752e-01,

In [ ]:
%cd '/content/drive/MyDrive/projects/Artenal/yolov5'

/content/drive/MyDrive/projects/Artenal/yolov5


In [ ]:
from PIL import Image
img=Image.open('/content/download.jpg')
img=img.resize((416,416))

In [ ]:
img.save('download.jpg')

In [ ]:
!python detect.py --weights yolov5s.pt --img 416 --conf 0.4 --source 'download.jpg' --save-crop

detect: weights=['yolov5s.pt'], source=download.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-297-g72a81e7 Python-3.7.13 torch-1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
image 1/1 /content/drive/MyDrive/projects/Artenal/yolov5/download.jpg: 416x416 4 oranges, Done. (0.009s)
Speed: 0.4ms pre-process, 9.2ms inference, 1.3ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp20
1 labels saved to runs/detect/exp20/labels
